In [1]:
import os

from dotenv import load_dotenv
load_dotenv()

GEMINI_API_KEY = os.getenv("GOOGLE_API_KEY")
HUGGINGFACE_API_KEY = os.getenv("HUGGINGFACE_TOKEN")


In [3]:
from langchain_community.document_loaders.csv_loader import CSVLoader


loader = CSVLoader(file_path=r'C:\Users\aethe\Desktop\Gen_AI\Google_Gemini\csv\pokemon.csv')
data = loader.load()

In [4]:
print(data)

[Document(page_content='#: 1\nName: Bulbasaur\nType 1: Grass\nType 2: Poison\nTotal: 318\nHP: 45\nAttack: 49\nDefense: 49\nSp. Atk: 65\nSp. Def: 65\nSpeed: 45\nGeneration: 1\nLegendary: False', metadata={'source': 'C:\\Users\\aethe\\Desktop\\Gen_AI\\Google_Gemini\\csv\\pokemon.csv', 'row': 0}), Document(page_content='#: 2\nName: Ivysaur\nType 1: Grass\nType 2: Poison\nTotal: 405\nHP: 60\nAttack: 62\nDefense: 63\nSp. Atk: 80\nSp. Def: 80\nSpeed: 60\nGeneration: 1\nLegendary: False', metadata={'source': 'C:\\Users\\aethe\\Desktop\\Gen_AI\\Google_Gemini\\csv\\pokemon.csv', 'row': 1}), Document(page_content='#: 3\nName: Venusaur\nType 1: Grass\nType 2: Poison\nTotal: 525\nHP: 80\nAttack: 82\nDefense: 83\nSp. Atk: 100\nSp. Def: 100\nSpeed: 80\nGeneration: 1\nLegendary: False', metadata={'source': 'C:\\Users\\aethe\\Desktop\\Gen_AI\\Google_Gemini\\csv\\pokemon.csv', 'row': 2}), Document(page_content='#: 3\nName: VenusaurMega Venusaur\nType 1: Grass\nType 2: Poison\nTotal: 625\nHP: 80\nAttack

In [6]:
from langchain.embeddings import HuggingFaceHubEmbeddings

embiddings = HuggingFaceHubEmbeddings(model="sentence-transformers/all-MiniLM-L6-v2")

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30)
text_chunkins = text_splitter.split_documents(data)
print(len(text_chunkins))

800


In [9]:
from langchain.vectorstores import Chroma
db = Chroma.from_documents(documents=text_chunkins,
                                 embedding=embiddings,
                                 persist_directory='chromo_db_csv')

In [17]:
# query it
query = "Oddish"
docs = db.similarity_search(query)

# print results
print(docs[0].page_content)

#: 368
Name: Gorebyss
Type 1: Water
Type 2: 
Total: 485
HP: 55
Attack: 84
Defense: 105
Sp. Atk: 114
Sp. Def: 75
Speed: 52
Generation: 3
Legendary: False


In [11]:
GEMINI_API_KEY = os.getenv('GOOGLE_API_KEY')

from langchain_google_genai import ChatGoogleGenerativeAI
LLM = ChatGoogleGenerativeAI(model="gemini-pro", convert_system_message_to_human=True)

In [12]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(llm=LLM, chain_type="stuff", retriever=db.as_retriever())

In [16]:
query = 'Explain about Oddish'
output = qa.run(query)
print(output)

Oddish is a Grass/Poison type Pokemon from the first generation. It has a total of 320, with 45 HP, 50 Attack, 55 Defense, 75 Sp. Atk, 65 Sp. Def, and 30 Speed.
